In [1]:
import gzip
import random
import scipy
from collections import defaultdict
# import tensorflow as tf
import csv
from implicit import bpr
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, dok_matrix

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    c = csv.reader(f)
    header = next(c)
    for l in c:
        d = dict(zip(header,l))
        yield d['user_id'],d['recipe_id'],d

In [3]:
## Bayesian Personalized Ranking

data = []
for user,recipe,d in readCSV("trainInteractions.csv.gz"):
    u = d['user_id']
    i = d['recipe_id']
    r = int(d['rating'])
    data.append(d)


# nTrain = int(len(data) * 0.6)
# data = data[:nTrain]
random.shuffle(data)

In [4]:
userIDs,itemIDs = {},{}

for d in data:
    u,i = d['user_id'],d['recipe_id']
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)

nUsers,nItems = len(userIDs),len(itemIDs)

nUsers,nItems

(13533, 151462)

In [5]:
Xiu = scipy.sparse.lil_matrix((nItems, nUsers))
for d in data:
    Xiu[itemIDs[d['recipe_id']],userIDs[d['user_id']]] = 1
    
Xui = scipy.sparse.csr_matrix(Xiu.T)

model = bpr.BayesianPersonalizedRanking(factors = 5)
model.fit(Xiu)

  0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
N=50000
predictions = open("predictions_Made.txt", 'w')
for l in open("stub_Made.txt"):
    if l.startswith("user_id"):
    #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if u in userIDs.keys() and  i in itemIDs.keys():
        u_index = userIDs[u]
        i_index = itemIDs[i]
    
        related = model.similar_items(u_index,N)
        related_index = [x[0] for x in related][1:]

        if i_index in related_index:
            predictions.write(u + '-' + i + ",1\n")
        else:
            predictions.write(u + '-' + i + ",0\n")
            
    else:
        predictions.write(u + '-' + i + ",0\n")
        
predictions.close()